# 02: IOU (Intersection over Union) - Hands-On Calculation

**Course:** Deep Neural Network Architectures (21CSE558T)  
**Module 5:** Object Detection  
**Week 13:** Object Localization and Bounding Boxes

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the IOU (Intersection over Union) formula
2. Calculate IOU from scratch for bounding boxes
3. Implement vectorized IOU for efficiency
4. Visualize overlapping boxes and interpret IOU values

**Estimated Time:** 15 minutes

---

## What is IOU?

**IOU (Intersection over Union)** is a fundamental metric in object detection that measures how well a predicted bounding box matches the ground truth bounding box.

### Why Do We Need IOU?

- **Quantify Overlap:** Measure the overlap between predicted and actual object locations
- **Evaluate Predictions:** Determine if a detection is a True Positive or False Positive
- **Non-Maximum Suppression:** Remove duplicate detections
- **Loss Function:** Train object detection models

### The Formula

```
IOU = Area(Intersection) / Area(Union)
    = Area(A ∩ B) / Area(A ∪ B)
```

### IOU Range

- **Range:** [0, 1]
- **0:** No overlap between boxes
- **1:** Perfect match (boxes are identical)

### Visual Representation

```
Box A:  ┌─────────┐
        │         │
        │    ┌────┼────┐  Box B
        │    │////│    │  (/// = Intersection)
        └────┼────┘    │
             │         │
             └─────────┘

Intersection = Overlapping area (////)
Union = Total area covered by both boxes
```

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

## Simple Visual Example

In [ ]:
# Define two overlapping boxes in XYXY format: [x_min, y_min, x_max, y_max]
box1 = [20, 30, 80, 90]  # Blue box (Ground Truth)
box2 = [50, 50, 120, 110]  # Red box (Prediction)

# Calculate intersection coordinates
x_min_inter = max(box1[0], box2[0])  # max(20, 50) = 50
y_min_inter = max(box1[1], box2[1])  # max(30, 50) = 50
x_max_inter = min(box1[2], box2[2])  # min(80, 120) = 80
y_max_inter = min(box1[3], box2[3])  # min(90, 110) = 90

# Calculate areas
inter_width = x_max_inter - x_min_inter  # 80 - 50 = 30
inter_height = y_max_inter - y_min_inter  # 90 - 50 = 40
intersection = inter_width * inter_height  # 30 * 40 = 1200

area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])  # 60 * 60 = 3600
area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])  # 70 * 60 = 4200

union = area1 + area2 - intersection  # 3600 + 4200 - 1200 = 6600

iou = intersection / union  # 1200 / 6600 ≈ 0.182

# Visualize
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Draw box1 (Ground Truth - Blue)
rect1 = patches.Rectangle((box1[0], box1[1]), box1[2]-box1[0], box1[3]-box1[1],
                          linewidth=3, edgecolor='blue', facecolor='none', label='Ground Truth')
ax.add_patch(rect1)

# Draw box2 (Prediction - Red)
rect2 = patches.Rectangle((box2[0], box2[1]), box2[2]-box2[0], box2[3]-box2[1],
                          linewidth=3, edgecolor='red', facecolor='none', label='Prediction')
ax.add_patch(rect2)

# Draw intersection (Green fill)
rect_inter = patches.Rectangle((x_min_inter, y_min_inter), inter_width, inter_height,
                               linewidth=2, edgecolor='green', facecolor='green', 
                               alpha=0.3, label='Intersection')
ax.add_patch(rect_inter)

# Add text annotations
ax.text(50, 15, f'IOU = {iou:.3f}', fontsize=16, fontweight='bold', 
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
ax.text(50, 5, f'Intersection = {intersection}', fontsize=12)
ax.text(50, -5, f'Union = {union}', fontsize=12)

ax.set_xlim(0, 140)
ax.set_ylim(-10, 120)
ax.set_aspect('equal')
ax.legend(loc='upper right', fontsize=12)
ax.set_title('IOU Calculation: Visual Example', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')

plt.tight_layout()
plt.show()

print(f"\nManual Calculation Results:")
print(f"Box 1 Area: {area1}")
print(f"Box 2 Area: {area2}")
print(f"Intersection Area: {intersection}")
print(f"Union Area: {union}")
print(f"IOU: {iou:.4f}")

## IOU Formula Implementation

In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate IOU between two boxes in XYXY format
    
    Args:
        box1: [x_min, y_min, x_max, y_max] - Ground truth or first box
        box2: [x_min, y_min, x_max, y_max] - Prediction or second box
    
    Returns:
        iou: float between 0 and 1
    """
    # Step 1: Calculate intersection coordinates
    # The intersection rectangle has:
    # - Top-left corner at the maximum of the two top-left corners
    # - Bottom-right corner at the minimum of the two bottom-right corners
    x_min_inter = max(box1[0], box2[0])
    y_min_inter = max(box1[1], box2[1])
    x_max_inter = min(box1[2], box2[2])
    y_max_inter = min(box1[3], box2[3])
    
    # Step 2: Calculate intersection area
    # Use max(0, ...) to handle non-overlapping boxes
    inter_width = max(0, x_max_inter - x_min_inter)
    inter_height = max(0, y_max_inter - y_min_inter)
    intersection = inter_width * inter_height
    
    # Step 3: Calculate individual box areas
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Step 4: Calculate union area
    # Union = Area1 + Area2 - Intersection
    # (We subtract intersection because it's counted twice in Area1 + Area2)
    union = area1 + area2 - intersection
    
    # Step 5: Calculate IOU
    # Handle edge case where union is 0 to avoid division by zero
    iou = intersection / union if union > 0 else 0
    
    return iou

# Test the function
test_box1 = [20, 30, 80, 90]
test_box2 = [50, 50, 120, 110]
test_iou = calculate_iou(test_box1, test_box2)

print(f"Test IOU: {test_iou:.4f}")
print("\nFunction implementation successful!")

## Test Cases

In [ ]:
# Define test cases
print("IOU Test Cases\n" + "="*50)

# Test Case 1: No overlap (IOU = 0.0)
box1_case1 = [10, 10, 50, 50]
box2_case1 = [60, 60, 100, 100]
iou_case1 = calculate_iou(box1_case1, box2_case1)
print(f"\nTest Case 1: No Overlap")
print(f"Box 1: {box1_case1}")
print(f"Box 2: {box2_case1}")
print(f"IOU: {iou_case1:.4f} (Expected: 0.0000)")

# Test Case 2: Partial overlap (~0.3)
box1_case2 = [20, 20, 80, 80]
box2_case2 = [50, 50, 110, 110]
iou_case2 = calculate_iou(box1_case2, box2_case2)
print(f"\nTest Case 2: Partial Overlap")
print(f"Box 1: {box1_case2}")
print(f"Box 2: {box2_case2}")
print(f"IOU: {iou_case2:.4f} (Expected: ~0.14)")

# Test Case 3: High overlap (~0.7)
box1_case3 = [20, 20, 100, 100]
box2_case3 = [30, 30, 110, 110]
iou_case3 = calculate_iou(box1_case3, box2_case3)
print(f"\nTest Case 3: High Overlap")
print(f"Box 1: {box1_case3}")
print(f"Box 2: {box2_case3}")
print(f"IOU: {iou_case3:.4f} (Expected: ~0.61)")

# Test Case 4: Perfect match (IOU = 1.0)
box1_case4 = [25, 25, 75, 75]
box2_case4 = [25, 25, 75, 75]
iou_case4 = calculate_iou(box1_case4, box2_case4)
print(f"\nTest Case 4: Perfect Match")
print(f"Box 1: {box1_case4}")
print(f"Box 2: {box2_case4}")
print(f"IOU: {iou_case4:.4f} (Expected: 1.0000)")

print("\n" + "="*50)
print("All test cases completed!")

## Visualization Function

In [ ]:
def visualize_iou(box1, box2, title="IOU Visualization"):
    """
    Draw boxes and show IOU value with visual representation
    
    Args:
        box1: [x_min, y_min, x_max, y_max] - Ground truth (blue)
        box2: [x_min, y_min, x_max, y_max] - Prediction (red)
        title: Title for the plot
    """
    # Calculate IOU
    iou = calculate_iou(box1, box2)
    
    # Calculate intersection coordinates
    x_min_inter = max(box1[0], box2[0])
    y_min_inter = max(box1[1], box2[1])
    x_max_inter = min(box1[2], box2[2])
    y_max_inter = min(box1[3], box2[3])
    
    inter_width = max(0, x_max_inter - x_min_inter)
    inter_height = max(0, y_max_inter - y_min_inter)
    
    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    # Draw box1 (Ground Truth - Blue)
    rect1 = patches.Rectangle((box1[0], box1[1]), box1[2]-box1[0], box1[3]-box1[1],
                              linewidth=3, edgecolor='blue', facecolor='blue', 
                              alpha=0.2, label='Ground Truth')
    ax.add_patch(rect1)
    
    # Draw box2 (Prediction - Red)
    rect2 = patches.Rectangle((box2[0], box2[1]), box2[2]-box2[0], box2[3]-box2[1],
                              linewidth=3, edgecolor='red', facecolor='red', 
                              alpha=0.2, label='Prediction')
    ax.add_patch(rect2)
    
    # Draw intersection (Green fill)
    if inter_width > 0 and inter_height > 0:
        rect_inter = patches.Rectangle((x_min_inter, y_min_inter), inter_width, inter_height,
                                       linewidth=2, edgecolor='green', facecolor='green', 
                                       alpha=0.5, label='Intersection')
        ax.add_patch(rect_inter)
    
    # Set plot limits with padding
    all_x = [box1[0], box1[2], box2[0], box2[2]]
    all_y = [box1[1], box1[3], box2[1], box2[3]]
    padding = 20
    ax.set_xlim(min(all_x) - padding, max(all_x) + padding)
    ax.set_ylim(min(all_y) - padding, max(all_y) + padding)
    
    # Add IOU value as title
    color = 'green' if iou > 0.5 else 'orange' if iou > 0.3 else 'red'
    ax.set_title(f"{title}\nIOU = {iou:.3f}", fontsize=14, fontweight='bold', color=color)
    
    ax.set_aspect('equal')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    
    plt.tight_layout()
    plt.show()

# Test the visualization function
print("Testing visualization function...")
visualize_iou([20, 30, 80, 90], [50, 50, 120, 110], "Sample IOU Visualization")

## Interactive Examples

In [ ]:
# Visualize all 4 test cases in a 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('IOU Examples: Different Overlap Scenarios', fontsize=16, fontweight='bold')

test_cases = [
    ([10, 10, 50, 50], [60, 60, 100, 100], "Case 1: No Overlap"),
    ([20, 20, 80, 80], [50, 50, 110, 110], "Case 2: Partial Overlap"),
    ([20, 20, 100, 100], [30, 30, 110, 110], "Case 3: High Overlap"),
    ([25, 25, 75, 75], [25, 25, 75, 75], "Case 4: Perfect Match")
]

for idx, (box1, box2, case_title) in enumerate(test_cases):
    ax = axes[idx // 2, idx % 2]
    
    # Calculate IOU
    iou = calculate_iou(box1, box2)
    
    # Calculate intersection
    x_min_inter = max(box1[0], box2[0])
    y_min_inter = max(box1[1], box2[1])
    x_max_inter = min(box1[2], box2[2])
    y_max_inter = min(box1[3], box2[3])
    inter_width = max(0, x_max_inter - x_min_inter)
    inter_height = max(0, y_max_inter - y_min_inter)
    
    # Draw boxes
    rect1 = patches.Rectangle((box1[0], box1[1]), box1[2]-box1[0], box1[3]-box1[1],
                              linewidth=2, edgecolor='blue', facecolor='blue', alpha=0.2)
    ax.add_patch(rect1)
    
    rect2 = patches.Rectangle((box2[0], box2[1]), box2[2]-box2[0], box2[3]-box2[1],
                              linewidth=2, edgecolor='red', facecolor='red', alpha=0.2)
    ax.add_patch(rect2)
    
    # Draw intersection if exists
    if inter_width > 0 and inter_height > 0:
        rect_inter = patches.Rectangle((x_min_inter, y_min_inter), inter_width, inter_height,
                                       linewidth=1, edgecolor='green', facecolor='green', alpha=0.5)
        ax.add_patch(rect_inter)
    
    # Set plot properties
    all_coords = box1 + box2
    padding = 10
    ax.set_xlim(min(all_coords[::2]) - padding, max(all_coords[::2]) + padding)
    ax.set_ylim(min(all_coords[1::2]) - padding, max(all_coords[1::2]) + padding)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    color = 'green' if iou > 0.5 else 'orange' if iou > 0.3 else 'red'
    ax.set_title(f"{case_title}\nIOU = {iou:.3f}", fontweight='bold', color=color)
    
    # Add legend to first subplot only
    if idx == 0:
        ax.legend([patches.Patch(facecolor='blue', alpha=0.2),
                  patches.Patch(facecolor='red', alpha=0.2),
                  patches.Patch(facecolor='green', alpha=0.5)],
                 ['Ground Truth', 'Prediction', 'Intersection'],
                 loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

print("\nIOU Values Summary:")
for idx, (box1, box2, case_title) in enumerate(test_cases):
    iou = calculate_iou(box1, box2)
    print(f"{case_title}: IOU = {iou:.4f}")

## Edge Cases

Let's explore some important edge cases in IOU calculation:

In [ ]:
print("IOU Edge Cases\n" + "="*60)

# Edge Case 1: Nested boxes (one completely inside another)
print("\nEdge Case 1: Nested Boxes (Small box inside larger box)")
box_outer = [10, 10, 100, 100]  # Large box
box_inner = [30, 30, 70, 70]    # Small box completely inside
iou_nested = calculate_iou(box_outer, box_inner)

area_outer = (box_outer[2] - box_outer[0]) * (box_outer[3] - box_outer[1])
area_inner = (box_inner[2] - box_inner[0]) * (box_inner[3] - box_inner[1])

print(f"Outer box: {box_outer}, Area = {area_outer}")
print(f"Inner box: {box_inner}, Area = {area_inner}")
print(f"IOU: {iou_nested:.4f}")
print(f"Note: IOU = {area_inner}/{area_outer} = {area_inner/area_outer:.4f}")
print("     Intersection = smaller box area, Union = larger box area")

visualize_iou(box_outer, box_inner, "Edge Case 1: Nested Boxes")

# Edge Case 2: Touching boxes (share edge, IOU = 0)
print("\nEdge Case 2: Touching Boxes (Share edge but no area overlap)")
box_left = [10, 10, 50, 50]
box_right = [50, 10, 90, 50]  # Shares right edge of left box
iou_touching = calculate_iou(box_left, box_right)

print(f"Left box: {box_left}")
print(f"Right box: {box_right}")
print(f"IOU: {iou_touching:.4f}")
print("Note: Boxes share an edge but have zero area overlap")

visualize_iou(box_left, box_right, "Edge Case 2: Touching Boxes")

# Edge Case 3: Identical boxes (IOU = 1.0)
print("\nEdge Case 3: Identical Boxes")
box_identical1 = [25, 35, 75, 85]
box_identical2 = [25, 35, 75, 85]
iou_identical = calculate_iou(box_identical1, box_identical2)

print(f"Box 1: {box_identical1}")
print(f"Box 2: {box_identical2}")
print(f"IOU: {iou_identical:.4f}")
print("Note: Intersection = Union, therefore IOU = 1.0")

visualize_iou(box_identical1, box_identical2, "Edge Case 3: Identical Boxes")

print("\n" + "="*60)
print("Edge case analysis complete!")

## Interpreting IOU Values

Understanding what different IOU thresholds mean in practice:

### General Guidelines

- **IOU > 0.5:** Generally considered a "good match" in object detection
- **IOU > 0.7:** Strong match, high confidence prediction
- **IOU > 0.9:** Excellent match, nearly perfect localization
- **IOU < 0.3:** Poor match, likely a false positive

### Common Thresholds in Object Detection Benchmarks

#### PASCAL VOC Challenge
- **Threshold:** IOU > 0.5 for True Positive
- **Rationale:** Balance between strict localization and practical detection
- **Usage:** Most widely used threshold in early object detection papers

#### MS COCO Challenge
- **Multiple Thresholds:** IOU > 0.5, 0.75, 0.95
- **AP@0.5:** Average Precision at IOU=0.5 (lenient)
- **AP@0.75:** Average Precision at IOU=0.75 (strict)
- **AP@[0.5:0.95]:** Average over IOU thresholds from 0.5 to 0.95 in 0.05 steps
- **Rationale:** Evaluate localization quality at different strictness levels

### Application-Specific Considerations

| Application | Recommended IOU | Reasoning |
|-------------|----------------|------------|
| Face Detection | > 0.5 | Rough localization sufficient |
| Medical Imaging | > 0.8 | High precision required |
| Autonomous Driving | > 0.7 | Safety-critical accurate bounds |
| General Object Detection | > 0.5 | Industry standard |

### Visual Interpretation

```
IOU = 0.1-0.3:  ▓░░░  Poor overlap
IOU = 0.3-0.5:  ▓▓░░  Moderate overlap  
IOU = 0.5-0.7:  ▓▓▓░  Good overlap (PASCAL VOC threshold)
IOU = 0.7-0.9:  ▓▓▓▓  Strong overlap (COCO strict threshold)
IOU = 0.9-1.0:  ▓▓▓▓  Excellent overlap
```

---

## Vectorized IOU (Advanced)

When working with multiple boxes, we can use NumPy broadcasting for efficient batch IOU calculation:

In [ ]:
def calculate_iou_batch(boxes1, boxes2):
    """
    Calculate IOU between N boxes and M boxes using NumPy broadcasting
    Much faster than loops for large number of boxes
    
    Args:
        boxes1: (N, 4) array in XYXY format
        boxes2: (M, 4) array in XYXY format
    
    Returns:
        iou_matrix: (N, M) array of IOU values
        where iou_matrix[i, j] = IOU between boxes1[i] and boxes2[j]
    """
    # Convert to numpy arrays
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)
    
    # Expand dimensions for broadcasting: (N, 1, 4) and (1, M, 4)
    boxes1_expanded = np.expand_dims(boxes1, axis=1)  # (N, 1, 4)
    boxes2_expanded = np.expand_dims(boxes2, axis=0)  # (1, M, 4)
    
    # Calculate intersection coordinates (N, M, 4)
    x_min_inter = np.maximum(boxes1_expanded[:, :, 0], boxes2_expanded[:, :, 0])
    y_min_inter = np.maximum(boxes1_expanded[:, :, 1], boxes2_expanded[:, :, 1])
    x_max_inter = np.minimum(boxes1_expanded[:, :, 2], boxes2_expanded[:, :, 2])
    y_max_inter = np.minimum(boxes1_expanded[:, :, 3], boxes2_expanded[:, :, 3])
    
    # Calculate intersection area (N, M)
    inter_width = np.maximum(0, x_max_inter - x_min_inter)
    inter_height = np.maximum(0, y_max_inter - y_min_inter)
    intersection = inter_width * inter_height
    
    # Calculate individual box areas (N,) and (M,)
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])
    
    # Broadcast to (N, M)
    area1 = area1[:, np.newaxis]  # (N, 1)
    area2 = area2[np.newaxis, :]  # (1, M)
    
    # Calculate union (N, M)
    union = area1 + area2 - intersection
    
    # Calculate IOU (N, M)
    iou_matrix = np.where(union > 0, intersection / union, 0)
    
    return iou_matrix

# Test vectorized implementation
print("Testing Vectorized IOU Implementation\n" + "="*60)

# Create sample boxes
ground_truths = np.array([
    [10, 10, 50, 50],
    [60, 60, 100, 100],
    [20, 20, 80, 80]
])

predictions = np.array([
    [15, 15, 55, 55],   # Close to GT 1
    [65, 65, 105, 105], # Close to GT 2
    [25, 25, 85, 85],   # Close to GT 3
    [40, 40, 90, 90]    # Between GT 1 and 3
])

print(f"Ground Truth boxes: {ground_truths.shape[0]}")
print(f"Predicted boxes: {predictions.shape[0]}")
print(f"\nComputing {ground_truths.shape[0]} × {predictions.shape[0]} = {ground_truths.shape[0] * predictions.shape[0]} IOU values...\n")

# Calculate IOU matrix
iou_matrix = calculate_iou_batch(ground_truths, predictions)

print("IOU Matrix (rows=GT, cols=Predictions):")
print("      Pred0   Pred1   Pred2   Pred3")
for i, row in enumerate(iou_matrix):
    print(f"GT{i}:  ", end="")
    for val in row:
        print(f"{val:6.3f}  ", end="")
    print()

# Find best matches
print("\nBest Matches:")
for i, gt_box in enumerate(ground_truths):
    best_pred_idx = np.argmax(iou_matrix[i])
    best_iou = iou_matrix[i, best_pred_idx]
    print(f"GT{i} ← Prediction{best_pred_idx} (IOU = {best_iou:.3f})")

print("\n" + "="*60)
print("Vectorized IOU calculation successful!")

## Performance Comparison

In [ ]:
import time

print("Performance Comparison: Loop vs Vectorized IOU\n" + "="*60)

# Generate random boxes
num_boxes = 100
np.random.seed(42)

# Random boxes in range [0, 100]
boxes1 = np.random.randint(0, 80, size=(num_boxes, 2))
boxes1 = np.concatenate([boxes1, boxes1 + np.random.randint(10, 30, size=(num_boxes, 2))], axis=1)

boxes2 = np.random.randint(0, 80, size=(num_boxes, 2))
boxes2 = np.concatenate([boxes2, boxes2 + np.random.randint(10, 30, size=(num_boxes, 2))], axis=1)

print(f"Testing with {num_boxes} boxes vs {num_boxes} boxes")
print(f"Total IOU calculations: {num_boxes * num_boxes} = {num_boxes**2:,}\n")

# Method 1: Loop-based (naive)
print("Method 1: Loop-based implementation")
start_time = time.time()
iou_loop = np.zeros((num_boxes, num_boxes))
for i in range(num_boxes):
    for j in range(num_boxes):
        iou_loop[i, j] = calculate_iou(boxes1[i], boxes2[j])
loop_time = time.time() - start_time
print(f"Time taken: {loop_time:.4f} seconds\n")

# Method 2: Vectorized
print("Method 2: Vectorized implementation")
start_time = time.time()
iou_vectorized = calculate_iou_batch(boxes1, boxes2)
vectorized_time = time.time() - start_time
print(f"Time taken: {vectorized_time:.4f} seconds\n")

# Verify results match
max_diff = np.max(np.abs(iou_loop - iou_vectorized))
print(f"Maximum difference between methods: {max_diff:.10f}")
print(f"Results match: {np.allclose(iou_loop, iou_vectorized)}\n")

# Performance summary
speedup = loop_time / vectorized_time
print("="*60)
print(f"SPEEDUP: {speedup:.1f}x faster with vectorization!")
print(f"Vectorized method is {speedup:.1f} times faster than loops")
print("="*60)

# Visualization of performance
fig, ax = plt.subplots(figsize=(10, 6))
methods = ['Loop-based', 'Vectorized']
times = [loop_time, vectorized_time]
colors = ['red', 'green']

bars = ax.bar(methods, times, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, time_val in zip(bars, times):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{time_val:.4f}s',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Time (seconds)', fontsize=12, fontweight='bold')
ax.set_title(f'IOU Calculation Performance ({num_boxes}×{num_boxes} = {num_boxes**2:,} calculations)\n'
             f'Speedup: {speedup:.1f}x', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Takeaway: Always use vectorized operations for batch processing!")

## Exercises

Test your understanding with these hands-on exercises:

---

### TODO Exercise 1: Manual IOU Calculation

Calculate the IOU for the following two bounding boxes **by hand** (show your work):

- Box A: `[10, 10, 50, 50]`
- Box B: `[30, 30, 70, 70]`

Steps:
1. Calculate intersection coordinates
2. Calculate intersection area
3. Calculate individual box areas
4. Calculate union area
5. Calculate IOU

Then verify your answer using the `calculate_iou()` function.

---

### TODO Exercise 2: IOU Threshold Question

What IOU value indicates that two bounding boxes do NOT overlap at all?

- A) IOU = 0.0
- B) IOU = 0.5
- C) IOU = 1.0
- D) IOU = -1.0

**Answer:** ___________

**Explanation:** ___________________________________________

---

### TODO Exercise 3: Best Match Selection

Given 1 ground truth box and 5 predicted boxes:

```python
ground_truth = [20, 20, 80, 80]

predictions = [
    [25, 25, 85, 85],   # Prediction 0
    [10, 10, 40, 40],   # Prediction 1  
    [50, 50, 100, 100], # Prediction 2
    [15, 15, 75, 75],   # Prediction 3
    [100, 100, 150, 150] # Prediction 4
]
```

**Task:** 
1. Calculate IOU between ground truth and each prediction
2. Find which prediction has the highest IOU
3. Visualize the best match

---

### TODO Exercise 4: XYWH Format IOU

Implement a function to calculate IOU for boxes in **XYWH format** (center_x, center_y, width, height):

```python
def calculate_iou_xywh(box1_xywh, box2_xywh):
    """
    Calculate IOU for boxes in XYWH format
    
    Args:
        box1_xywh: [center_x, center_y, width, height]
        box2_xywh: [center_x, center_y, width, height]
    
    Returns:
        iou: float between 0 and 1
    """
    # TODO: Convert XYWH to XYXY format
    # Hint: x_min = center_x - width/2, x_max = center_x + width/2
    
    # TODO: Use calculate_iou() function
    
    pass

# Test your function
box1_xywh = [50, 50, 40, 40]  # center at (50,50), size 40×40
box2_xywh = [60, 60, 40, 40]  # center at (60,60), size 40×40

# TODO: Calculate and print IOU
```

---

In [ ]:
# Exercise 1: Manual Calculation Verification
print("Exercise 1: Verify Your Manual Calculation\n" + "="*60)

box_a = [10, 10, 50, 50]
box_b = [30, 30, 70, 70]

# TODO: Show your manual calculation work here as comments
# Step 1: Intersection coordinates
# x_min_inter = max(10, 30) = ?
# y_min_inter = max(10, 30) = ?
# x_max_inter = min(50, 70) = ?
# y_max_inter = min(50, 70) = ?

# Step 2: Intersection area
# inter_width = ? - ? = ?
# inter_height = ? - ? = ?
# intersection = ? × ? = ?

# Step 3: Individual areas
# area_a = (50-10) × (50-10) = ?
# area_b = (70-30) × (70-30) = ?

# Step 4: Union
# union = ? + ? - ? = ?

# Step 5: IOU
# iou = ? / ? = ?

# Verify with function
iou_calculated = calculate_iou(box_a, box_b)
print(f"Function result: IOU = {iou_calculated:.4f}")
print("\nDid your manual calculation match? ___________")

In [ ]:
# Exercise 3: Best Match Selection
print("Exercise 3: Find Best Prediction Match\n" + "="*60)

ground_truth = [20, 20, 80, 80]

predictions = [
    [25, 25, 85, 85],     # Prediction 0
    [10, 10, 40, 40],     # Prediction 1  
    [50, 50, 100, 100],   # Prediction 2
    [15, 15, 75, 75],     # Prediction 3
    [100, 100, 150, 150]  # Prediction 4
]

# TODO: Calculate IOU for each prediction
# TODO: Find the best match
# TODO: Visualize the best match

print("TODO: Complete this exercise")

In [ ]:
# Exercise 4: XYWH Format Implementation
print("Exercise 4: IOU for XYWH Format\n" + "="*60)

def calculate_iou_xywh(box1_xywh, box2_xywh):
    """
    Calculate IOU for boxes in XYWH format
    
    Args:
        box1_xywh: [center_x, center_y, width, height]
        box2_xywh: [center_x, center_y, width, height]
    
    Returns:
        iou: float between 0 and 1
    """
    # TODO: Convert XYWH to XYXY format
    # Hint: x_min = center_x - width/2, x_max = center_x + width/2
    
    pass

# Test cases
box1_xywh = [50, 50, 40, 40]  # center at (50,50), size 40×40
box2_xywh = [60, 60, 40, 40]  # center at (60,60), size 40×40

# TODO: Calculate and print IOU
print("TODO: Implement this function")

## Summary

### Key Takeaways

1. **IOU Formula:** Intersection / Union measures overlap between bounding boxes

2. **IOU Range:** [0, 1] where:
   - 0 = no overlap
   - 1 = perfect match
   - > 0.5 = generally good match

3. **Implementation Steps:**
   - Calculate intersection coordinates
   - Calculate intersection area (handle non-overlap with max(0, ...))
   - Calculate union = area1 + area2 - intersection
   - IOU = intersection / union

4. **Vectorization:** For batch processing, use NumPy broadcasting instead of loops (10-50x speedup)

5. **Applications:**
   - Evaluating object detection predictions
   - Non-Maximum Suppression (NMS)
   - Training loss functions
   - Benchmark metrics (PASCAL VOC, MS COCO)

### What's Next?

**Notebook 03:** Bounding Box Visualization
- Drawing boxes on images
- Color-coding by confidence/class
- Visualizing detections with labels

---

**Great work!** You now understand IOU, a fundamental metric in object detection.

---